In [1]:
pwd

'C:\\Users\\kosmo-19\\Desktop\\code_box\\project\\project_team(보험약관)'

In [2]:
import scipy as sp
import numpy as np
import pandas as pd

from konlpy.tag import Kkma     ; kkma = Kkma()
from konlpy.tag import Hannanum ;hannanum = Hannanum()
from konlpy.tag import Okt      ; t = Okt()   # 구 트위터에서 만든 라이브러리
import pickle

In [3]:
from sklearn.feature_extraction.text import CountVectorizer  # 빈도수
from sklearn.feature_extraction.text import TfidfVectorizer  # 통계적인 툴
from sklearn.linear_model import LogisticRegression  # 씨그모이드가 이거임/ 0 또는 1
from sklearn.pipeline import Pipeline  # 모델을 한꺼번에 저장해두고 사용
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_excel('sentence(가공5).xlsx')
print(df.shape)
df

(636, 2)


,sentence,label
0,무조건 60% 미만은 안 내려 가요,1
1,저희 회사를 믿으셔도 돼요,1
2,선 이자 드릴게요,1
3,이자를 은행권보다 더 줄 수 있는거죠,1
4,모든 암이 보장된다고 보시면 돼죠,1
5,제기한 민원 때문에 내부적으로 징계를 받게 될 것 같아서요,1
6,그저 보장성으로 생활자금을 받으려 했다고 말하면 되거든요,1
7,"가입 확인 차 본사에서 전화가 오는데, 연금용으로 가입한 것이 아니라고 말해주셔야 돼요",1
8,이 상품은 연금 전환 기능도 있다. 혹자는 무조건 상품 이름에 '연금'이라는 단어가...,1
9,지금 매달 70만원씩 적금만 들고 있다. 더 효율적인 저축을 위해서 저축 기간에 따...,1


In [8]:
pd.__version__

'0.23.4'

In [9]:
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)

In [10]:
def step1_data_preprocessing() :
    # 수집한 데이터를 읽어오기
    df = pd.read_excel('sentence(270).xlsx')
    
    # 리스트로 바꿔줘
    text_list = df['sentence'].tolist()
    star_list = df['label'].tolist()
    
    # 학습셋, 테스트셋 분리
    from sklearn.model_selection import train_test_split
    
    # 데이터의 80%는 학습셋이고 20%는 테스트셋
    text_train, text_test, star_train, star_test = \
    train_test_split(text_list, star_list, test_size=0.2, random_state=0)
    
    return text_train, text_test, star_train, star_test

In [11]:
step1_data_preprocessing()

(['보험료 납입주기는 일시납, 월납, 3개월납, 6개월납, 연납입니다. ',
  '지금 매달 70만원씩 적금만 들고 있다. 더 효율적인 저축을 위해서 저축 기간에 따라 단기, 중기, 장기 상품을 각각 가입해야 한다. 장기저축을 위해서는 연금에 가입하는 것이 좋다',
  '추가납입보험료는 기본보험료 이외에 계약일 이후 1년 경과 후부터 보험만기 2년전 계약해당일가지(납입후 잔여보험기간이 2년 이하인 경우는 납입기간 종료시점까지) 납입한도 내에서 수시로 납입하는 보험료입니다.',
  '재물 및 배상책임 관련 보장: 타인을 위한 계약을 체결하는 경우에 그 타인의 위임이 없을 때에는 반드시 이를 회사에 알려야 하며, 이를 알리지 않았을 때에는 그 타인은 이 계약이 체결된 사실을 알지 못하였다는 사유로 회사에 이의를 제기할 수 없으며, 보험사고가 발생한 경우에 계약자가 그 타인에게 보험사고의 발생으로 생긴 손해를 배상한 때에는 계약자는 그 타인의 권리를 해하지 않는 범위 안에서 회사에 보험금의 지급을 청구할 수 있습니다.',
  '상해입원의 상급병실료 차액은 실제로 사용한 병실과 기준병실의 병실료 차액 에서 50%를 뺀 금액입니다. (단, 1일 평균금액 10만원 한도)',
  '할아버지, 비교안내확인서에 확인서명 직접하시러 오기 불편하니까 제가 대신 해드릴게요.',
  '정진수 너 이번에 취업했다매.. 내가 진짜 이런 부탁 처음인거 알지? 나 다니는 충무로대리점에 가서 보험 하나만 들어주면 안될까? 수당 나오면 반 줄게',
  '이게 저축성 보험이랑 비슷한거예요',
  '이 상품 손실없이 100% 보장받으실 수 있어요. 노후대비용 연금이라고 보시면 됩니다. ',
  '타업계 보면 아시겠지만 이 상품만큼 완벽한 상품은 더더욱 없어요.',
  '이건 형식적인거거든요',
  '예정사업비율이란, 보험회사가 보험계약의 체결, 유지 및 관리 등에 필요한 경비로 사용 하기 위하여 보험료 중 일정비율을 비용으로 책정한 것을 말합니다.',
  '고객님 진짜 오늘 상품가입하시면, 고객

In [12]:
def step2_learning(X_train, y_train, X_test, y_test) :
    
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)  #클래스 등장

    logistic = LogisticRegression(C=10, penalty='l2', random_state=0)

    pipe = Pipeline([('vect', tfidf),('clf', logistic)])
    

    pipe.fit(X_train, y_train)
    

    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    

    with open('pipe.dat', 'wb') as fp: 
        pickle.dump(pipe, fp)  
        
    print('저장 완료') 

In [13]:
def learning() :   # 감성분석 모델 생성
    text_train, text_test, star_train, star_test = step1_data_preprocessing()
    step2_learning(text_train, star_train, text_test, star_test)

In [14]:
learning()

0.9814814814814815
저장 완료


In [15]:
def step3_using_model() :
    # 저장된 모델 불러오기
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)
        
    while True :
        text = input('보험설계사의 음성을 분석하고 있습니다 : ')  # 인풋

        str = [text]
        
        r1 = np.max(pipe.predict_proba(str)*100)
        # 예측 결과
        r2 = pipe.predict(str)[0]
        
        if r2 == 1 : 
            print('보험설계사의 설명은 불완전판매로 의심됩니다.')

        else :
            print('해당 계약을 추진하셔도 좋습니다')
            
        print('정확도 : %.3f' %r1)

In [16]:
# 사용함수
def using() :      # 감성분석 모델 사용
    step3_using_model()

In [ ]:
using()

보험설계사의 음성을 분석하고 있습니다 : 저만 믿으세요 할머님
보험설계사의 설명은 불완전판매로 의심됩니다.
정확도 : 94.824
보험설계사의 음성을 분석하고 있습니다 : 특별히 이번만 해드릴게요
보험설계사의 설명은 불완전판매로 의심됩니다.
정확도 : 97.962
보험설계사의 음성을 분석하고 있습니다 :  그 다음에 나머지 별도로 가입을 하시는 방법이 있어요 고객님 같은 경우에는 지금 현재 7,500만 가입을 하시게 되면
보험설계사의 설명은 불완전판매로 의심됩니다.
정확도 : 67.584
보험설계사의 음성을 분석하고 있습니다 :  최근에 치료를 받으셨거나 약을 드시거나 하시는 거 있으신가요
보험설계사의 설명은 불완전판매로 의심됩니다.
정확도 : 76.191
보험설계사의 음성을 분석하고 있습니다 : 어떻게 되시네요  고객님 지금 취업준비생 하시는 거는 학원 다니시거나 하시는 건 없으신가요  학원 생으로 등록을 하고 보험료를 말씀 드릴께요  고객님 실비는 고객님 아프거나 다쳐서 병원에 가서 치료를 받고 내셨던 치료를 받으시는 보험인데요  그렇기 때문에 이제 거의 모든 상해 날 질병 회장이 되세요  혹시 5년이나 입원 수술 하셨거나 최근에 치료를 받으셨거나 양을 드시거나 하시는 거 있으신가요  전혀 있으신가요  전혀 없으시고요 하면 그런 거예요 고객님 택배 최근에 내가 감기 로라도 병원에 갔다 오신 이력이 있으신지 를 여쭤보는 거예요  치료를 한두 번 받으셨거나 이런 것도 다 해당되시는 거거든요  병원 감기나 배탈 때문에 몇 번 간 적은 앞에 차 폐차하려면  일단 그 상해나 질병으로 만약에 병원에 입원을 하시게 되면 1사고당 5천만 원 한도 내에서 고객님 내셨던 치료비 중에 의료보험이 되는지요 비는  10% 의료 보험이 안 되는 비급여 치료비는 20% 를 공제하고 나머지를 돌려받게 되실 것 때문에  초원으로 가시게 되면 두 가지를 비교합니다  하나는 병원 규모별로 1만 원은 만 원 준종합병원 15,000원 종합병원 2만 원을 공제하거나  일본 되는 거 10% 안 되